# BarcodeSeqKit Core

> Core data structures for barcode extraction from sequencing data

In [1]:
#| default_exp core

In [2]:
#| hide
from nbdev.showdoc import *

## Introduction

BarcodeSeqKit is a library designed for extracting and processing barcoded sequences from next-generation sequencing data. This notebook contains the core data structures and utility functions for the library.

In [3]:
#| export
import os
import re
import sys
import gzip
import json
import enum
import logging
from pathlib import Path
from dataclasses import dataclass, field
from typing import List, Dict, Tuple, Set, Optional, Union, Iterator, Any
from Bio.Seq import Seq

## Enumerations
Let's define the basic enumerations used throughout the library.

In [4]:
#| export
class OrientationType(enum.Enum):
    """Orientation types for barcode sequences."""
    FORWARD = "FR"
    REVERSE_COMPLEMENT = "RC"
    ANY = "ANY"

In [ ]:
#| export
class BarcodeLocationType(enum.Enum):
    """Location types for barcodes."""
    FIVE_PRIME = "5"
    THREE_PRIME = "3"
    UNKNOWN = "UNK"

In [7]:
#| export
class FileFormat(enum.Enum):
    """Supported file formats."""
    BAM = "BAM"
    FASTQ = "FASTQ"
    UNKNOWN = "UNKNOWN"

    @classmethod
    def detect_format(cls, file_path: str) -> 'FileFormat':
        """Detect the format of a file based on its extension.
        
        Args:
            file_path: Path to the file
            
        Returns:
            The detected file format
        """
        lower_path = file_path.lower()
        if lower_path.endswith('.bam'):
            return cls.BAM
        elif any(lower_path.endswith(ext) for ext in ['.fastq', '.fq', '.fastq.gz', '.fq.gz']):
            return cls.FASTQ
        else:
            return cls.UNKNOWN

## Core Data Structures
Now let's define the main data structures for barcode configuration and matching.

In [6]:
#| export
@dataclass
class BarcodeConfig:
    """Configuration for a barcode sequence."""
    sequence: str
    location: BarcodeLocationType = BarcodeLocationType.UNKNOWN
    name: Optional[str] = None
    description: Optional[str] = None
    
    def __post_init__(self):
        # Clean the sequence and ensure uppercase
        self.sequence = self.sequence.strip().upper()
        # If no name is provided, use the sequence as the name
        if self.name is None:
            self.name = self.sequence
    
    @property
    def reverse_complement(self) -> str:
        """Return the reverse complement of the barcode sequence."""
        return str(Seq(self.sequence).reverse_complement())
    
    def to_dict(self) -> Dict[str, Any]:
        """Convert the barcode configuration to a dictionary."""
        return {
            "sequence": self.sequence,
            "location": self.location.value,
            "name": self.name,
            "description": self.description,
            "reverse_complement": self.reverse_complement
        }

NameError: name 'BarcodeLocationType' is not defined

In [8]:
#| export
@dataclass
class BarcodeMatch:
    """Represents a match of a barcode in a sequence."""
    barcode: BarcodeConfig
    orientation: OrientationType
    position: int
    sequence: str
    
    def __str__(self) -> str:
        """String representation of the barcode match."""
        return f"{self.barcode.name} ({self.orientation.value}) at position {self.position}"

NameError: name 'BarcodeConfig' is not defined

## Extraction Statistics
Let's define a class to track extraction statistics without relying on the abstract extractor.

In [ ]:
#| export
@dataclass
class ExtractionStatistics:
    """Statistics collected during barcode extraction."""
    total_reads: int = 0
    total_barcode_matches: int = 0
    matches_by_barcode: Dict[str, int] = field(default_factory=dict)
    matches_by_orientation: Dict[str, int] = field(default_factory=dict)
    matches_by_category: Dict[str, int] = field(default_factory=dict)
    no_barcode_count: int = 0
    
    def update_barcode_match(self, barcode_match: BarcodeMatch, category: Optional[str] = None) -> None:
        """Update statistics based on a barcode match.
        
        Args:
            barcode_match: The barcode match
            category: The category (e.g., 'barcode5_orientFR'). If None, will be derived from the match.
        """
        self.total_barcode_matches += 1
        
        # Update matches by barcode
        barcode_name = barcode_match.barcode.name
        if barcode_name not in self.matches_by_barcode:
            self.matches_by_barcode[barcode_name] = 0
        self.matches_by_barcode[barcode_name] += 1
        
        # Update matches by orientation
        orientation = barcode_match.orientation.value
        if orientation not in self.matches_by_orientation:
            self.matches_by_orientation[orientation] = 0
        self.matches_by_orientation[orientation] += 1
        
        # Derive category if not provided
        if category is None:
            # Try to derive a reasonable category
            location = barcode_match.barcode.location.value
            if location in ["5", "3"]:
                category = f"barcode{location}_orient{orientation}"
            else:
                category = f"barcode_orient{orientation}"
        
        # Update matches by category
        if category not in self.matches_by_category:
            self.matches_by_category[category] = 0
        self.matches_by_category[category] += 1
    
    def to_dict(self) -> Dict[str, Any]:
        """Convert the statistics to a dictionary."""
        return {
            "total_reads": self.total_reads,
            "total_barcode_matches": self.total_barcode_matches,
            "matches_by_barcode": self.matches_by_barcode,
            "matches_by_orientation": self.matches_by_orientation,
            "matches_by_category": self.matches_by_category,
            "no_barcode_count": self.no_barcode_count,
            "match_rate": (self.total_barcode_matches / self.total_reads) if self.total_reads > 0 else 0
        }
    
    def save_json(self, output_path: str) -> None:
        """Save the statistics to a JSON file."""
        with open(output_path, 'w') as f:
            json.dump(self.to_dict(), f, indent=2)
    
    def save_tsv(self, output_path: str) -> None:
        """Save the statistics to a TSV file."""
        with open(output_path, 'w') as f:
            # Write header
            f.write("Metric\tValue\n")
            
            # Write overall statistics
            stats_dict = self.to_dict()
            f.write(f"TotalReads\t{stats_dict['total_reads']}\n")
            f.write(f"TotalBarcodeMatches\t{stats_dict['total_barcode_matches']}\n")
            f.write(f"NoBarcodeCount\t{stats_dict['no_barcode_count']}\n")
            f.write(f"MatchRate\t{stats_dict['match_rate']:.4f}\n")
            
            # Write barcode-specific statistics
            f.write("\nBarcode\tCount\n")
            for barcode, count in stats_dict['matches_by_barcode'].items():
                f.write(f"{barcode}\t{count}\n")
            
            # Write orientation-specific statistics
            f.write("\nOrientation\tCount\n")
            for orientation, count in stats_dict['matches_by_orientation'].items():
                f.write(f"{orientation}\t{count}\n")
            
            # Write category-specific statistics
            f.write("\nCategory\tCount\n")
            for category, count in stats_dict['matches_by_category'].items():
                f.write(f"{category}\t{count}\n")

## Configuration Handling
Let's create a simplified configuration class without the extractor abstraction.

In [ ]:
#| export
@dataclass
class BarcodeSeqKitConfig:
    """Configuration for barcode extraction operations."""
    barcodes: List[BarcodeConfig]
    output_prefix: str
    output_dir: str = "."
    merge_orientations: bool = False
    # Removed keep_unmatched parameter to simplify logic
    verbose: bool = False
    log_file: Optional[str] = None
    max_mismatches: int = 0
    search_softclipped: bool = False  # For BAM processing
    search_both_reads: bool = False   # For paired FASTQ processing
    compress_output: bool = True      # For FASTQ output
    
    def __post_init__(self):
        # Validate barcodes
        if not self.barcodes:
            raise ValueError("At least one barcode must be provided")
        
        # Ensure output directory exists
        os.makedirs(self.output_dir, exist_ok=True)
        
        # Setup logging
        self.logger = self._setup_logging()
    
    def _setup_logging(self) -> logging.Logger:
        """Set up logging configuration."""
        logger = logging.getLogger("BarcodeSeqKit")
        logger.setLevel(logging.DEBUG if self.verbose else logging.INFO)
        
        # Clear existing handlers to prevent duplicates
        logger.handlers = []
        
        # Console handler
        console_handler = logging.StreamHandler()
        console_handler.setLevel(logging.INFO)
        console_formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
        console_handler.setFormatter(console_formatter)
        logger.addHandler(console_handler)
        
        # File handler (if requested)
        if self.log_file:
            file_handler = logging.FileHandler(self.log_file)
            file_handler.setLevel(logging.DEBUG)
            file_formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
            file_handler.setFormatter(file_formatter)
            logger.addHandler(file_handler)
        
        # Prevent propagation to root logger
        logger.propagate = False
        
        return logger
    
    @classmethod
    def from_dict(cls, config_dict: Dict[str, Any]) -> 'BarcodeSeqKitConfig':
        """Create a configuration from a dictionary."""
        # Extract barcode configurations
        barcode_configs = []
        for barcode_dict in config_dict.get("barcodes", []):
            location = BarcodeLocationType(barcode_dict.get("location", "UNK"))
            barcode_configs.append(BarcodeConfig(
                sequence=barcode_dict["sequence"],
                location=location,
                name=barcode_dict.get("name"),
                description=barcode_dict.get("description")
            ))
        
        # Create and return the configuration
        return cls(
            barcodes=barcode_configs,
            output_prefix=config_dict["output_prefix"],
            output_dir=config_dict.get("output_dir", "."),
            merge_orientations=config_dict.get("merge_orientations", False),
            keep_unmatched=config_dict.get("keep_unmatched", True),
            verbose=config_dict.get("verbose", False),
            log_file=config_dict.get("log_file"),
            max_mismatches=config_dict.get("max_mismatches", 0),
            search_softclipped=config_dict.get("search_softclipped", False),
            search_both_reads=config_dict.get("search_both_reads", False),
            compress_output=config_dict.get("compress_output", True)
        )
    
    def to_dict(self) -> Dict[str, Any]:
        """Convert the configuration to a dictionary."""
        return {
            "barcodes": [barcode.to_dict() for barcode in self.barcodes],
            "output_prefix": self.output_prefix,
            "output_dir": self.output_dir,
            "merge_orientations": self.merge_orientations,
            "keep_unmatched": self.keep_unmatched,
            "verbose": self.verbose,
            "log_file": self.log_file,
            "max_mismatches": self.max_mismatches,
            "search_softclipped": self.search_softclipped,
            "search_both_reads": self.search_both_reads,
            "compress_output": self.compress_output
        }
    
    def save_yaml(self, output_path: str) -> None:
        """Save the configuration to a YAML file."""
        import yaml
        with open(output_path, 'w') as f:
            yaml.dump(self.to_dict(), f, default_flow_style=False)
    
    @classmethod
    def load_yaml(cls, input_path: str) -> 'BarcodeSeqKitConfig':
        """Load configuration from a YAML file."""
        import yaml
        with open(input_path, 'r') as f:
            config_dict = yaml.safe_load(f)
        return cls.from_dict(config_dict)

## Basic Utility Functions
Let's add some basic utility functions for common operations.

In [ ]:
#| export
def save_stats_and_config(stats: ExtractionStatistics, config: BarcodeSeqKitConfig) -> None:
    """Save extraction statistics and configuration to files.
    
    Args:
        stats: Extraction statistics
        config: Extraction configuration
    """
    # Save statistics
    json_path = os.path.join(config.output_dir, f"{config.output_prefix}_extraction_stats.json")
    tsv_path = os.path.join(config.output_dir, f"{config.output_prefix}_extraction_stats.tsv")
    stats.save_json(json_path)
    stats.save_tsv(tsv_path)
    config.logger.info(f"Statistics saved to {json_path} and {tsv_path}")
    
    # Save configuration
    yaml_path = os.path.join(config.output_dir, f"{config.output_prefix}_config.yaml")
    config.save_yaml(yaml_path)
    config.logger.info(f"Configuration saved to {yaml_path}")

In [ ]:
#| export
def print_extraction_summary(stats: ExtractionStatistics, config: BarcodeSeqKitConfig) -> None:
    """Print a summary of extraction statistics to the log.
    
    Args:
        stats: Extraction statistics
        config: Extraction configuration
    """
    logger = config.logger
    
    logger.info("Extraction complete")
    logger.info(f"Total reads: {stats.total_reads}")
    logger.info(f"Barcode matches: {stats.total_barcode_matches}")
    logger.info(f"No barcode: {stats.no_barcode_count}")
    
    if stats.total_reads > 0:
        match_rate = (stats.total_barcode_matches / stats.total_reads) * 100
        logger.info(f"Match rate: {match_rate:.2f}%")
    
    # Log barcode-specific statistics
    for barcode_name, count in stats.matches_by_barcode.items():
        logger.info(f"Barcode {barcode_name}: {count} matches")
    
    # Log orientation-specific statistics
    for orientation, count in stats.matches_by_orientation.items():
        logger.info(f"Orientation {orientation}: {count} matches")
    
    # Log category-specific statistics
    if stats.matches_by_category:
        logger.info("Match statistics by category:")
        for category, count in stats.matches_by_category.items():
            logger.info(f"  {category}: {count} matches")

## Example Usage
Let's demonstrate how to use these core components.

In [ ]:
# Example of creating a barcode configuration
barcode_5prime = BarcodeConfig(
    sequence="TCGCGAGGC",
    location=BarcodeLocationType.FIVE_PRIME,
    name="5",
    description="5' barcode for phenotyping experiment"
)

barcode_3prime = BarcodeConfig(
    sequence="GGCCGGCCGG",
    location=BarcodeLocationType.THREE_PRIME,
    name="3",
    description="3' barcode for phenotyping experiment"
)

# Create a configuration
config = BarcodeSeqKitConfig(
    barcodes=[barcode_5prime, barcode_3prime],
    output_prefix="example",
    output_dir="./output",
    merge_orientations=False,
    verbose=True
)

# Print the configuration
print(f"Barcode 5' sequence: {barcode_5prime.sequence}")
print(f"Barcode 5' reverse complement: {barcode_5prime.reverse_complement}")
print(f"Barcode 3' sequence: {barcode_3prime.sequence}")
print(f"Barcode 3' reverse complement: {barcode_3prime.reverse_complement}")

# Save the configuration
config.save_yaml("example_config.yaml")
print("Configuration saved to example_config.yaml")

## Conclusion
This notebook establishes the core data structures for the BarcodeSeqKit library without relying on abstract classes and factory patterns. It provides all the necessary components for barcode configuration, matching, and statistics tracking, which will be used by the specialized processors for BAM and FASTQ files.

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()